# Predicting Thyroid Cancer Recurrence - Model Building

### Purpose of this notebook
To explore the models that were used for this study, optimise feature selection and find the most performant models.

## 1. Imports and Loading data

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# SciKit-learn imports
from sklearn.model_selection import train_test_split


In [ ]:
file_path = "../data/raw/Thyroid_Diff.csv"
df = pd.read_csv(file_path)